<a href="https://colab.research.google.com/github/bill7845/colab_repo/blob/master/dacon_%EC%A0%9C%EC%A3%BC%EC%B9%B4%EB%93%9C%EC%82%AC%EC%9A%A9%EB%9F%89%EC%98%88%EC%B8%A1/modeling_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
data = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/201901-202003.csv', encoding='utf-8')
data.shape

(24697792, 12)

In [3]:
data.head(1)

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4


In [4]:
## data preprocessing

data = data.fillna('세종') # 결측값 ''로 대체

## 날짜 처리
def grap_year(data): # year 
    data = str(data)
    return int(data[:4])

def grap_month(data): # month 
    data = str(data)
    return int(data[4:])

data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1) # 기존 날짜컬럼 삭제
data.head()

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019,1
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019,1
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019,1
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019,1
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019,1


In [5]:
# 결측치 확인
data.isnull().sum()

CARD_SIDO_NM    0
CARD_CCG_NM     0
STD_CLSS_NM     0
HOM_SIDO_NM     0
HOM_CCG_NM      0
AGE             0
SEX_CTGO_CD     0
FLC             0
CSTMR_CNT       0
AMT             0
CNT             0
year            0
month           0
dtype: int64

In [6]:
# 시군구도 필요없으니 시군구 변수도 삭제
del data['CARD_CCG_NM']
del data['HOM_CCG_NM']
del data['CSTMR_CNT']
del data['CNT']

In [7]:
## SEX_CTGE_CD 인코딩
sex_condi = {1:0, 2:1} # 남자 0 여자 1로 인코딩
data['SEX_CTGO_CD'] = data['SEX_CTGO_CD'].map(sex_condi)

## 지역민인지 여부 변수 추가해보기
def get_local_nlocal(df):
  if df['CARD_SIDO_NM'] == df['HOM_SIDO_NM']:
    return 1 # 지역민 1
  else :
    return 0 # 외지인 0

data['local_nlocal'] = data.apply(get_local_nlocal,axis=1)

# 컬럼 순서
data = data[['year','month','CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM','AGE','SEX_CTGO_CD','FLC','local_nlocal','AMT']]

In [8]:
data.head()

,year,month,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,local_nlocal,AMT
0,2019,1,강원,건강보조식품 소매업,강원,20s,0,1,1,311200
1,2019,1,강원,건강보조식품 소매업,강원,30s,0,2,1,1374500
2,2019,1,강원,건강보조식품 소매업,강원,30s,1,2,1,818700
3,2019,1,강원,건강보조식품 소매업,강원,40s,0,3,1,1717000
4,2019,1,강원,건강보조식품 소매업,강원,40s,0,4,1,1047300


* 지역별 모델링

In [9]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from itertools import product

## encoding def
## 인코딩 완료된 df형태로 반환
def encoder_func(df):
  dtypes = df.dtypes
  encoders = {}
  for column in df.columns:
    if str(dtypes[column]) == 'object':
      encoder = LabelEncoder()
      encoder.fit(df[column])
      encoders[column] = encoder

  for column in encoders.keys():
    encoder = encoders[column]
    df[column] = encoder.transform(df[column])

  return df,encoders


# 지역별 모델링
def modeling(full_data): # full_data는 전체 데이터의미. (data)

  res_model_dict = {} # 각 지역별 모델 저장할 딕셔너리
  res_forecasting = pd.DataFrame(columns=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT']) # res_forecating에 각 지역별 예측결과를 누적하여 저장.

  city_list = full_data['CARD_SIDO_NM'].unique() # 전체 지역 리스트
  
  for city in city_list : # 각 지역별로 반복함.
    print(city) 
    sampling = full_data.query("CARD_SIDO_NM == @city") # 전체 데이터에서 한 지역씩 sampling 변수에 담음
    Nonexistent_std_clss = list( set(full_data['STD_CLSS_NM'].unique()) - set(sampling['STD_CLSS_NM'].unique())) # 각 지역별로 집계값이 0인 업종들
    sampling,encoders = encoder_func(sampling) # 위에서 정의한 인코딩함수에 sampling을 넣어 인코딩 된 데이터와 인코더 객체를 반환받음

    train_num = sampling.sample(frac=1, random_state=0) # 데이터 셔플
    train_features = train_num.drop(['CARD_SIDO_NM','AMT'],axis=1) # features # 각 지역별로 진행하기 때문에 CARD_SIDO_NM은 제외함.
    train_target = np.log1p(train_num['AMT']) # target

    model = RandomForestRegressor(n_jobs=-1, random_state=0) # model 선언 
    model.fit(train_features, train_target) # 학습

    res_model_dict['model_'+city] = model # 학습 된 모델을 res_model 딕셔너리에 저장

    ## 템플릿 생성 부분
    ## 각 지역별로 진행하기 때문에 CARD_SIDO_NM은 제외함.
    year_temp = [2020]
    month_temp = [4,7]
    STD_CLSS_NM_temp = sampling['STD_CLSS_NM'].unique()
    HOM_SIDO_NM_temp = sampling['HOM_SIDO_NM'].unique()
    AGE_temp = sampling['AGE'].unique()
    SEX_CTGO_CD_temp = sampling['SEX_CTGO_CD'].unique()
    FLC_temp = sampling['FLC'].unique()
    local_nlocal_temp = sampling['local_nlocal'].unique() # 이외에 변수 추가시 같은 방법으로 추가해주면 됨
    comb_list = [year_temp, month_temp, STD_CLSS_NM_temp, HOM_SIDO_NM_temp, AGE_temp, SEX_CTGO_CD_temp, FLC_temp, local_nlocal_temp]
    forecasting_temp = np.array(list(product(*comb_list)))
    forecasting_temp = pd.DataFrame(data=forecasting_temp, columns=train_features.columns) # 예측 템플릿 완성

    pred = model.predict(forecasting_temp) # 예측
    pred = np.expm1(pred)
    forecasting_temp['AMT'] = np.round(pred, 0) 
    forecasting_temp['REG_YYMM'] = forecasting_temp['year']*100 + forecasting_temp['month']
    forecasting_temp = forecasting_temp[['REG_YYMM','STD_CLSS_NM', 'AMT']]
    forecasting_temp.insert(1,'CARD_SIDO_NM',city) # 첫번째 열에 "CARD_SIDO_NM"이라는 열을 추가함 값은 해당 지역이름임
    forecasting_temp = forecasting_temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False) # 지역별,업종별 월간 사용금액 합계
    forecasting_temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(forecasting_temp['STD_CLSS_NM']) # STD_CLSS_NM inverse encoding
    
    ## 각 지역별로 집계값이 0인 업종들이 있음
    ## submission에서는 이 업종들도 포함해서 제출해야 하기 때문에 0으로 때려넣을것임
    for clss in Nonexistent_std_clss:
      for i in range(2):
        mm = [202004,202007]
        tmp = pd.DataFrame([[ mm[i],city,clss,0 ]],columns=forecasting_temp.columns)
        forecasting_temp = pd.concat([forecasting_temp,tmp]) # forecasting_temp에 concat
    
    res_forecasting = pd.concat([res_forecasting,forecasting_temp]) # res_forecasting에 지역별 예측완료 된 값들을 누적하여 저장
    
  return res_model_dict,res_forecasting # 지역별 모델 딕셔너리, 최종 예측값 반환

res_model_dict,res_forecasting = modeling(data) # 실행

강원
경기
경남
경북
광주
대구
대전
부산
서울
세종
울산
인천
전남
전북
제주
충남
충북


In [10]:
res_model_dict # 지역별 모델 

{'model_강원': RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=-1, oob_score=False,
                       random_state=0, verbose=0, warm_start=False),
 'model_경기': RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=-1, oob_score=False,
                       random_state=0, verbos

In [11]:
# submisson과 최종 예측 프레임 합치기
submission = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(res_forecasting, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
# submission.to_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/submission/NOTE_2.csv', encoding='utf-8-sig') # 저장 경로 설정 후 실행
submission

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,6.380596e+08
1,202004,강원,골프장 운영업,1.043502e+09
2,202004,강원,과실 및 채소 소매업,4.591908e+08
3,202004,강원,관광 민예품 및 선물용품 소매업,1.733519e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.898053e+08
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.642776e+08
1390,202007,충북,한식 음식점업,4.520023e+08
1391,202007,충북,호텔업,2.693059e+08
